In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import seaborn as sns

In [4]:
sns.set_context('poster', font_scale=1.25)

In [5]:
sns.set_style('ticks')

In [6]:
import expansion_interactions as ei

In [7]:
ei.sc

0.2933673469387755

In [10]:
strain_names = ['mCherry', 'eCFP', 'Black', 'eYFP']
zeta_measured = [0, .10, .19, .18]

In [12]:

num_strains = len(zeta_measured)

# Derive relative growth rates relative to the first strain
velocity_sq_ratios = np.zeros((num_strains, num_strains), dtype=np.double)
for i in range(num_strains):
    for j in range(num_strains):
        ui_sq_div_ref = 1 + zeta_measured[i]**2/4.
        uj_sq_div_ref = 1 + zeta_measured[j]**2/4.
        ui_sq_div_uj_sq = ui_sq_div_ref/uj_sq_div_ref
        velocity_sq_ratios[i, j] = ui_sq_div_uj_sq

vw_div_u = np.sqrt(velocity_sq_ratios-1)

In [13]:
vw_div_u

array([[ 0.        ,         nan,         nan,         nan],
       [ 0.05      ,  0.        ,         nan,         nan],
       [ 0.095     ,  0.08067669,  0.        ,  0.03029138],
       [ 0.09      ,  0.07473978,         nan,  0.        ]])

Ok, this actually looks reasonable...why didn't I do this earlier? It would have saved me a lot of time...

We now get the dataframe we want.

In [18]:
import pandas as pd

In [19]:
Dw = 0.092
u = 1.15
Ro = 3.50
Ro_star = 1.84
Dg = .56

# For superdiffusive walk
Db = 0.13
b = 1.53

In [35]:
all_data = []
for i in range(num_strains):
    for j in range(num_strains):
        cur_vw_div_u = vw_div_u[i, j]
        if np.isfinite(cur_vw_div_u) and (cur_vw_div_u != 0):
            data_list = []
            
            vw = cur_vw_div_u * u
            Ls = u*Dw/vw**2

            kappa = np.sqrt(Ro/Ls)

            sweeper = strain_names[i]
            sweepee = strain_names[j]
            
            data_list.append(sweeper)
            data_list.append(sweepee)
            velocity_ratio = np.sqrt(velocity_sq_ratios[i, j])
            data_list.append(velocity_ratio)
            data_list.append(vw)
            data_list.append(Ls)
            data_list.append(kappa)
            
            all_data.append(data_list)

In [36]:
interact_df = pd.DataFrame(data=all_data, columns=['Sweeper',
                                                        'Sweepee',
                                                        'Velocity Ratios',
                                                        'vw',
                                                        'Ls',
                                                        'kappa'])

In [37]:
interact_df.sort_values('Ls', inplace=True)

In [38]:
interact_df

,Sweeper,Sweepee,Velocity Ratios,vw,Ls,kappa
1,Black,mCherry,1.004502,0.109250,8.864266,0.455604
4,eYFP,mCherry,1.004042,0.103500,9.876543,0.431625
2,Black,eCFP,1.003249,0.092778,12.291188,0.386912
5,eYFP,eCFP,1.002789,0.085951,14.321429,0.358439
0,eCFP,mCherry,1.001249,0.057500,32.000000,0.239792
3,Black,eYFP,1.000459,0.034835,87.187027,0.145272


Wow. According to this, their velocity ratios are *so* small.

Ok. Yes. A different measurement of $D_s$ really clears up a lot of the issues here...good. I'm not sure if I need a competition experiment. It may help to clear up issues as well, IDK. But, the issue is, things do not necessarily behave the same way on a plate as in a liquid...so it may not help me. Yeah.

# Superdiffusive substitution

In [40]:
(ei.u*ei.Db/.14**2)**(2)

58.17953456892959